# MMVCの学習に必要なconfig系Fileを作成する

ver.2022/09/16

MMVC学習用の自身の音声データを録音します。


​

In [ ]:
#@title ## 1 Google Driveをマウント
#@markdown **このノートブックで、Google Driveを使用するための設定です。**

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#@markdown 　「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown 成功すれば、下記メッセージが出ます。 { display-mode: "form" }

#@markdown ``` 
#@markdown Mounted at /content/drive/
#@markdown ```

from google.colab import drive
drive.mount('/content/drive')

​

In [ ]:
#@title 2 MMVC_Trainerディレクトリに移動
#@markdown ​マウントしたGoogle DriveのMMVC_Trainerディレクトリに移動します。

#@markdown Google DriveでMMVC_Trainerの場所を確認し、以下でパスを指定してください。

#@markdown 正しいパスが指定されていれば、以下のようなメッセージが表示されます。

#@markdown ```
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown ```
#@markdown


#@markdown ​
#@markdown ### Settings
directory = "/content/drive/MyDrive/MMVC_Trainer" #@param {type:"string"}

%cd $directory
!ls -1

​

In [ ]:
#@title ## 3 録音の準備
#@markdown 録音用のプログラムを読み込みます。

base_dir = "./dataset/textful/00_myvoice/"
sampling_rate = 24000

import os
myvoice_dir = base_dir + "wav/"
mytext_dir = base_dir + "text/"
os.makedirs(myvoice_dir, exist_ok=True)
os.makedirs(mytext_dir, exist_ok=True)

# from https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
# from https://colab.research.google.com/github/espnet/notebook/blob/master/espnet2_asr_realtime_demo.ipynb

from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, filename='audio.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(filename, 'wb+') as f:
    f.write(b)

import librosa
import librosa.display
import matplotlib.pyplot as plt
import pysndfile
from IPython.display import display, Audio
import warnings
warnings.filterwarnings('ignore')

def rec(sec, filename, text, hira):
  if sec > 15 :
    print("15秒以上の音声は学習できません。\n録音秒数を15秒以下に指定してください。")
    return
  print(f"{text}\n{hira}")
  record(sec, "temp.wav")
  print("---終了---")
  speech, rate = librosa.load("temp.wav", sr=sampling_rate)
  speecht, index = librosa.effects.trim(speech, top_db=30)
  pysndfile.sndio.write(myvoice_dir + filename + ".wav", speecht, rate=rate, format='wav', enc='pcm16')
  with open(mytext_dir + filename + ".txt", 'w') as mytext:
    mytext.write(hira)
  librosa.display.waveplot(speecht, sr=rate)
  plt.show()
  display(Audio(speecht, rate=rate))


​

## 4 学習用音声を録音する

ITAコーパスを利用して音声を録音します。  
以下の100件の学習データを録音します。  
実行したら話す内容が表示されるので「---終了---」と表示されるまでに録音してください。  
録音した波形が表示されるので、再生してみて良ければ次の録音をします。  
うまく録音できていなかったら再度実行して録音しなおしてください。  
時々録音状態が終了しない時があり、その際は左の■で止めて、ページ自体をスーパーリロード（shiftキーを押しながら再読み込み）してから再度実行してください。  
録音時間が短い場合、先頭「rec(5, …」などの「5」の部分が録音秒数になっているので、修正してください。ただしMMVCの最大学習時間が15秒となっているのでそれ以下に指定してください。  
学習データは MMVC_Trainer/dataset/textful/00_myvoice/wav/ 以下に保存されます。

In [ ]:
rec(3, "emotion001", "えっ嘘でしょ。", "えっうそでしょ。")

In [ ]:
rec(5, "emotion002", "シュヴァイツァーは見習うべき人間です。", "しゅゔぁいつぁーわみならうべきにんげんです。")

In [ ]:
rec(5, "emotion003", "デーヴィスさんはとても疲れているように見える。", "でーゔぃすさんわとてもつかれているよーにみえる。")

In [ ]:
rec(5, "emotion004", "スティーヴはジェーンから手紙をもらった。", "すてぃーゔわじぇーんからてがみをもらった。")

In [ ]:
rec(10, "emotion005", "彼女はモーツァルトやベートーヴェンといった、古典派の作曲家が好きだ。", "かのじょわもーつぁるとやべーとーゔぇんといった、こてんはのさっきょくかがすきだ。")

In [ ]:
rec(10, "emotion006-1", "ストラットフォード・オン・エイヴォンは、シェイクスピアの生まれたところですが", "すとらっとふぉーどおんえいゔぉんわ、しぇいくすぴあのうまれたところですが")

In [ ]:
rec(5, "emotion006-2", "毎年多くの観光客が訪れます。", "まいとしおおくのかんこーきゃくがおとずれます。")

In [ ]:
rec(5, "emotion007", "彼はピューマを慣らすのに、大いに骨を折った。", "かれわぴゅーまをならすのに、おおいにほねをおった。")

In [ ]:
rec(5, "emotion008", "彼が解雇されるとは妙な話だ。", "かれがかいこされるとわみょーなはなしだ。")

In [ ]:
rec(5, "emotion009", "クリスはヴァンパイア・ナイトを倒した。", "くりすわゔぁんぱいあないとをたおした。")

In [ ]:
rec(5, "emotion010", "彼のあだ名は言い得て妙だよね。", "かれのあだなわいいえてみょーだよね。")

In [ ]:
rec(7, "emotion011-1", "イタリア旅行で彼は、いくつか景勝の地として有名な都市", "いたりありょこーでかれわ、いくつかけーしょーのちとしてゆうめーなとし")

In [ ]:
rec(7, "emotion011-2", "例えば、ナポリやフィレンツェを訪れた。", "たとえば、なぽりやふぃれんつぇをおとずれた。")

In [ ]:
rec(10, "emotion012", "ゼロという概念は、ヒンドゥー文化に由来している。", "ぜろとゆうがいねんわ、ひんどぅーぶんかにゆらいしている。")

In [ ]:
rec(5, "emotion013", "そのいたずら娘は何食わぬ顔をした。", "そのいたずらむすめわなにくわぬかおをした。")

In [ ]:
rec(10, "emotion014", "スミスさん、ピエール・デュボワをご紹介しますわ。私の親友なの。", "すみすさん、ぴえーるでゅぼわをごしょーかいしますわ。わたしのしんゆうなの。")

In [ ]:
rec(10, "emotion015", "どーすんの、このお店。完ッ全に閑古鳥が鳴いちゃってるじゃない。", "どーすんの、このおみせ。かんっぜんにかんこどりがないちゃってるじゃない。")

In [ ]:
rec(5, "emotion016", "頂上に着くと皆でヤッホーと叫んだ。", "ちょーじょーにつくとみんなでやっほーとさけんだ。")

In [ ]:
rec(5, "emotion017", "あっベルが鳴ってる。", "あっべるがなってる。")

In [ ]:
rec(5, "emotion018", "彼女は彼にぴょこんとおじぎをした。", "かのじょわかれにぴょこんとおじぎをした。")

In [ ]:
rec(7, "emotion019", "男子生徒のめいめいが、帽子に学校のバッジをつけています。", "だんしせーとのめーめーが、ぼーしにがっこーのばっじをつけています。")

In [ ]:
rec(3, "emotion020", "ヒューズが飛んだ。", "ひゅーずがとんだ。")

In [ ]:
rec(5, "emotion021", "私はポピュラー音楽を聞きたい。", "わたしわぽぴゅらーおんがくをききたい。")

In [ ]:
rec(5, "emotion022", "猫はにゃーにゃーと鳴く。", "ねこわにゃーにゃーとなく。")

In [ ]:
rec(10, "emotion023", "私の一番上の兄が父の名代としてその会合に出席した。", "わたしのいちばんうえのあにがちちのみょーだいとしてそのかいごーにしゅっせきした。")

In [ ]:
rec(5, "emotion024", "彼は首相にインタビューした。", "かれわしゅしょーにいんたびゅーした。")

In [ ]:
rec(10, "emotion025", "その会社の急速な成長は、その独特な戦略によるものだった。", "そのかいしゃのきゅうそくなせーちょーわ、そのどくとくなせんりゃくによるものだった。")

In [ ]:
rec(7, "emotion026", "私はいつもミネラルウォーターを持ち歩いています。", "わたしわいつもみねらるうぉーたーをもちあるいています。")

In [ ]:
rec(10, "emotion027", "彼女はハンドバッグを開けて家の鍵を探してみたが、見つからなかった。", "かのじょわはんどばっぐをあけていえのかぎをさがしてみたが、みつからなかった。")

In [ ]:
rec(7, "emotion028", "彼女はスタッフをまとめていけると思いますか？", "かのじょわすたっふをまとめていけるとおもいますか？")

In [ ]:
rec(10, "emotion029", "牛乳はあなたの体に良いだろう、毎日飲んだほうがいい。", "ぎゅうにゅうわあなたのからだによいだろー、まいにちのんだほーがいい。")

In [ ]:
rec(10, "emotion030", "あなたは流感になりかけているか、もっと重い病気かもしれません。", "あなたわりゅうかんになりかけているか、もっとおもいびょーきかもしれません。")

In [ ]:
rec(10, "emotion031", "彼女はその事件を、生き生きとした筆致で描写した。", "かのじょわそのじけんを、いきいきとしたひっちでびょーしゃした。")

In [ ]:
rec(10, "emotion032", "彼は自らの生涯を、インドでの病人の治療に捧げるつもりだ。", "かれわみずからのしょーがいを、いんどでのびょーにんのちりょーにささげるつもりだ。")

In [ ]:
rec(10, "emotion033", "奇妙な男で、彼は人から話し掛けられないと口をきかない。", "きみょーなおとこで、かれわひとからはなしかけられないとくちをきかない。")

In [ ]:
rec(7, "emotion034", "我々はその山脈で土着のガイドを雇った。", "われわれわそのさんみゃくでどちゃくのがいどをやとった。")

In [ ]:
rec(7, "emotion035", "彼女は息子に家で行儀よくするように言った。", "かのじょわむすこにいえでぎょーぎよくするよーにいった。")

In [ ]:
rec(7, "emotion036", "彼はライフルを拾い上げ、それで標的をねらった。", "かれわらいふるをひろいあげ、それでひょーてきをねらった。")

In [ ]:
rec(5, "emotion037", "私はこの本に八百円を払った。", "わたしわこのほんにはっぴゃくえんをはらった。")

In [ ]:
rec(10, "emotion038", "気分が悪くて入院したが、結果的にはたいしたことはなかった。", "きぶんがわるくてにゅういんしたが、けっかてきにわたいしたことわなかった。")

In [ ]:
rec(5, "emotion039-1", "アフィ狙いの釣り記事ですね。", "あふぃねらいのつりきじですね。")

In [ ]:
rec(10, "emotion039-2", "英語関係のコミュのあちこちにマルチポストしています。", "えーごかんけーのこみゅのあちこちにまるちぽすとしています。")

In [ ]:
rec(10, "emotion040", "トラベラーズチェックを現金に替えてくれるのはこの窓口ですか？", "とらべらーずちぇっくをげんきんにかえてくれるのわこのまどぐちですか？")

In [ ]:
rec(7, "emotion041", "本日の映画は、サウンド・オブ・ミュージックでございます。", "ほんじつのえーがわ、さうんどおぶみゅーじっくでございます。")

In [ ]:
rec(10, "emotion042", "概してわれわれ日本人は、外国語を話すのに少し臆病すぎる。", "がいしてわれわれにほんじんわ、がいこくごをはなすのにすこしおくびょーすぎる。")

In [ ]:
rec(5, "emotion043-1", "店の人は私のことを知るようになり", "みせのひとわわたしのことをしるよーになり")

In [ ]:
rec(7, "emotion043-2", "私はいつも同じウェイトレスに応対してもらっていた。", "わたしわいつもおなじうぇいとれすにおーたいしてもらっていた。")

In [ ]:
rec(5, "emotion044", "この丘からは何百万という星が見える。", "このおかからわなんびゃくまんとゆうほしがみえる。")

In [ ]:
rec(10, "emotion045", "彼女は大学に入学したら、親から経済的に独立しようと思っていた。", "かのじょわだいがくににゅうがくしたら、おやからけーざいてきにどくりつしよーとおもっていた。")

In [ ]:
rec(7, "emotion046", "サブマリンのペリスコープが、水中からにょっきり突き出ていた。", "さぶまりんのぺりすこーぷが、すいちゅうからにょっきりつきでていた。")

In [ ]:
rec(7, "emotion047", "別にブルマに特別な関心があるわけじゃない。", "べつにぶるまにとくべつなかんしんがあるわけじゃない。")

In [ ]:
rec(5, "emotion048", "そういうフェティシズムはないと思う。", "そーゆうふぇてぃしずむわないとおもう。")

In [ ]:
rec(7, "emotion049-1", "ウッド夫人が作ってくれるおいしい田舎風の料理を食べたし", "うっどふじんがつくってくれるおいしいいなかふうのりょーりをたべたし")

In [ ]:
rec(5, "emotion049-2", "ミルクをたくさん飲みました。", "みるくをたくさんのみました。")

In [ ]:
rec(7, "emotion050-1", "私の妻が瓶をわってしまったので", "わたしのつまがびんをわってしまったので")

In [ ]:
rec(7, "emotion050-2", "台所の床は牛乳だらけになっている。", "だいどころのゆかわぎゅうにゅうだらけになっている。")

In [ ]:
rec(10, "emotion051", "喧嘩をしていた二人の子供は、お互いにしかめっ面をして座っていた。", "けんかをしていたふたりのこどもわ、おたがいにしかめっつらをしてすわっていた。")

In [ ]:
rec(7, "emotion052-1", "事業を継続しながら、事業が依拠している不動産を", "じぎょーをけーぞくしながら、じぎょーがいきょしているふどーさんを")

In [ ]:
rec(7, "emotion052-2", "切り売りしていくことなど非現実的なのだ。", "きりうりしていくことなどひげんじつてきなのだ。")

In [ ]:
rec(7, "emotion053-1", "しかしペパーバーグは、そのオウムを研究することによって", "しかしぺぱーばーぐわ、そのおーむをけんきゅうすることによって")

In [ ]:
rec(7, "emotion053-2", "動物に対する考え方が変わったと言っている。", "どーぶつにたいするかんがえかたがかわったといっている。")

In [ ]:
rec(10, "emotion054", "パックマンがある条件を満たすと、追ってくるモンスターを逆襲して食べることができる。", "ぱっくまんがあるじょーけんをみたすと、おってくるもんすたーをぎゃくしゅうしてたべることができる。")

In [ ]:
rec(7, "emotion055-1", "その数百年後に、フォークが西洋のテーブルに現れることになるが", "そのすうひゃくねんごに、ふぉーくがせーよーのてーぶるにあらわれることになるが")

In [ ]:
rec(5, "emotion055-2", "ただちに受け入れられたわけではなかった。", "ただちにうけいれられたわけでわなかった。")

In [ ]:
rec(10, "emotion056", "トップのリーダーは犬の行動学ではアルファと呼ばれ、以下ベータ、ガンマと続きます。", "とっぷのりーだーわいぬのこーどーがくでわあるふぁとよばれ、いかべーた、がんまとつづきます。")

In [ ]:
rec(7, "emotion057", "ウィーンまでは歩くとどのくらいかかりますか？", "うぃーんまでわあるくとどのくらいかかりますか？")

In [ ]:
rec(5, "emotion058", "すみません、この辺に詳しくないんです。", "すみません、このへんにくわしくないんです。")

In [ ]:
rec(10, "emotion059-1", "目標は授業設計をするときの、学生の思考を触発するメディア教材の選択および", "もくひょーわじゅぎょーせっけーをするときの、がくせーのしこーをしょくはつするめでぃあきょーざいのせんたくおよび")

In [ ]:
rec(5, "emotion059-2", "活用方法について理解することである。", "かつよーほーほーについてりかいすることである。")

In [ ]:
rec(10, "emotion060", "とりあえず店の前、掃除しといてくれ。打ち水も頼む。", "とりあえずみせのまえ、そーじしといてくれ。うちみずもたのむ。")

In [ ]:
rec(10, "emotion061", "人々はトーナメントが始まる何ヶ月も前に、これらの入場券を買う。", "ひとびとわとーなめんとがはじまるなんかげつもまえに、これらのにゅうじょーけんをかう。")

In [ ]:
rec(10, "emotion062", "彼女の魅力は言葉では表現できない、とその芸術家は叫んだ。", "かのじょのみりょくわことばでわひょーげんできない、とそのげーじゅつかわさけんだ。")

In [ ]:
rec(10, "emotion063-1", "事象として簡単なことを、いかにも難しそうに表現する人は", "じしょーとしてかんたんなことを、いかにもむずかしそーにひょーげんするひとわ")

In [ ]:
rec(5, "emotion063-2", "あまり頭がよさそうではない。", "あまりあたまがよさそーでわない。")

In [ ]:
rec(7, "emotion064-1", "デザインも、アーチ型のロゴデザインにより", "でざいんも、あーちがたのろごでざいんにより")

In [ ]:
rec(7, "emotion064-2", "現代的で登場感、躍動感あるものに仕上げました。", "げんだいてきでとーじょーかん、やくどーかんあるものにしあげました。")

In [ ]:
rec(7, "emotion065", "そんなに慌てて運転して、一体どこへ行こうってんだよ。", "そんなにあわててうんてんして、いったいどこえいこーってんだよ。")

In [ ]:
rec(7, "emotion066", "時間はあるんだから、安全運転してくれよ。", "じかんわあるんだから、あんぜんうんてんしてくれよ。")

In [ ]:
rec(10, "emotion067-1", "ディスプレイはモニタともいい、コンピュータなどの機器から出力される静止画", "でぃすぷれいわもにたともいい、こんぴゅーたなどのききからしゅつりょくされるせーしが")

In [ ]:
rec(7, "emotion067-2", "または動画の映像信号を表示する機器である。", "またわどーがのえーぞーしんごーをひょーじするききである。")

In [ ]:
rec(7, "emotion068-1", "撃ち合いが少し静まったとき、パパが走ってフラットに行って", "うちあいがすこししずまったとき、ぱぱがはしってふらっとにいって")

In [ ]:
rec(5, "emotion068-2", "私たちにサンドイッチを持ってきてくれたわ。", "わたしたちにさんどいっちをもってきてくれたわ。")

In [ ]:
rec(7, "emotion069-1", "ギリシャのフットボールの試合では、一方のチームの選手は", "ぎりしゃのふっとぼーるのしあいでわ、いっぽーのちーむのせんしゅわ")

In [ ]:
rec(10, "emotion069-2", "相手チームの陣地のラインの向こう側にボールを持ち込もうとしたのです。", "あいてちーむのじんちのらいんのむこーがわにぼーるをもちこもーとしたのです。")

In [ ]:
rec(10, "emotion070-1", "泳者のシンディー・ニコラスは、へとへとになって泳ぎ切った後、ドーバーでかろうじて陸に上がってきたが", "えーしゃのしんでぃーにこらすわ、へとへとになっておよぎきったあと、どーばーでかろーじてりくにあがってきたが")

In [ ]:
rec(10, "emotion070-2", "海峡水泳協会のスポークスマンは、彼女がとても元気であると発表した。", "かいきょーすいえーきょーかいのすぽーくすまんわ、かのじょがとてもげんきであるとはっぴょーした。")

In [ ]:
rec(10, "emotion071", "若い男女が人里離れた洋館で恐怖の一夜を過ごすという、ホラーの定番スタイルだ。", "わかいだんじょがひとざとはなれたよーかんできょーふのいちやをすごすとゆう、ほらーのてーばんすたいるだ。")

In [ ]:
rec(10, "emotion072-1", "一つには、西洋人ではない人々が自分たち独自の文化に誇りを持ち始めてきたためと、また一つには、", "ひとつにわ、せーよーじんでわないひとびとがじぶんたちどくじのぶんかにほこりをもちはじめてきたためと、またひとつにわ")

In [ ]:
rec(10, "emotion072-2", "フォークを使わないそれらの地域は最も高い出生率の地域を抱えているという理由から、このことは当たっている。", "ふぉーくをつかわないそれらのちいきわもっともたかいしゅっしょーりつのちいきをかかえているとゆうりゆうから、このことわあたっている。")

In [ ]:
rec(7, "emotion073", "私たちは恐怖の中で生きていて、苦しんでいるの。", "わたしたちわきょーふのなかでいきていて、くるしんでいるの。")

In [ ]:
rec(10, "emotion074", "太陽も花も楽しめないし、私たちの子供時代も楽しめないのよ。", "たいよーもはなもたのしめないし、わたしたちのこどもじだいもたのしめないのよ。")

In [ ]:
rec(10, "emotion075-1", "弊社のエンジニアが日本国内で販売されている同様の製品と仕様を比較した結果", "へーしゃのえんじにあがにほんこくないではんばいされているどーよーのせーひんとしよーをひかくしたけっか")

In [ ]:
rec(7, "emotion075-2", "非常に競合力があると判断いたしました。", "ひじょーにきょーごーりょくがあるとはんだんいたしました。")

In [ ]:
rec(7, "emotion076-1", "ノースウエスタン大学の研究者、アイリーン・ペパーバーグは", "のーすうえすたんだいがくのけんきゅうしゃ、あいりーんぺぱーばーぐわ")

In [ ]:
rec(10, "emotion076-2", "オウムは人の口まねをするだけでなく言葉の意味を学ぶことができることを発見しつつある。", "おーむわひとのくちまねをするだけでなくことばのいみをまなぶことができることをはっけんしつつある。")

In [ ]:
rec(5, "emotion077", "名前をテョといいます。", "なまえをてょといいます。")

In [ ]:
rec(5, "emotion078", "ぴゅうぴゅう風が吹きこんでくる。", "ぴゅうぴゅうかぜがふきこんでくる。")

In [ ]:
rec(5, "emotion079", "キェー。ギェー。イェーイ。", "きぇー。ぎぇー。いぇーい。")

In [ ]:
rec(5, "emotion080", "ひぇーん。びぇーん。ぴぇーん。", "ひぇーん。びぇーん。ぴぇーん。")

In [ ]:
rec(5, "emotion081", "プリミェー。ミェルン。ニェン。", "ぷりみぇー。みぇるん。にぇん。")

In [ ]:
rec(5, "emotion082", "ステューデント。ダブルデュアル。", "すてゅーでんと。だぶるでゅある。")

In [ ]:
rec(7, "emotion083", "エテュード。エデュケーショナル。サブスティテューション。", "えてゅーど。えでゅけーしょなる。さぶすてぃてゅーしょん。")

In [ ]:
rec(3, "emotion084", "ジャデャクシュ。", "じゃでゃくしゅ。")

In [ ]:
rec(3, "emotion085", "ブレンドデョート。", "ぶれんどでょーと。")

In [ ]:
rec(5, "emotion086", "イデュスルファーゼ。", "いでゅするふぁーぜ。")

In [ ]:
rec(3, "emotion087", "あっあの。", "あっあの。")

In [ ]:
rec(3, "emotion088", "いっいえ。", "いっいえ。")

In [ ]:
rec(3, "emotion089", "えっえぇ。", "えっえぇ。")

In [ ]:
rec(3, "emotion090", "おっおい。", "おっおい。")

In [ ]:
rec(3, "emotion091", "んーとね。", "んーとね。")

In [ ]:
rec(3, "emotion092", "アンツィオ。", "あんつぃお。")

In [ ]:
rec(3, "emotion093", "エンツォ。", "えんつぉ。")

In [ ]:
rec(3, "emotion094", "カートゥーン。", "かーとぅーん。")

In [ ]:
rec(3, "emotion095", "スィーディー。", "すぃーでぃー。")

In [ ]:
rec(3, "emotion096", "ズィーブラ。", "ずぃーぶら。")

In [ ]:
rec(5, "emotion097", "デピュティーガバナー。", "でぴゅてぃーがばなー。")

In [ ]:
rec(3, "emotion098", "エリュシオン。", "えりゅしおん。")

In [ ]:
rec(3, "emotion099", "ガリェント。", "がりぇんと。")

In [ ]:
rec(3, "emotion100", "ラーテャン。", "らーてゃん。")